In [ ]:
import random
import queue
import concurrent.futures
from threading import Barrier

def share(secret, client_num):
    """
    Producing share based on the number of clients.
    """
    shares = [0] * client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def CP(i, x, y, U, V, W, input_queue, output_queue, barrier):
    """
    The main function each party executes separately.
    """
    
    def mul_number(i, s, d):
        """
        Multiplication function based on Beaver triple.
        """
        D = s - U
        E = d - V

        # Sending and receiving intermediate results
        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += (D_sum * E_sum)
        return Z

    return mul_number(i, x, y)

def SPDZ_prepare(beta1, beta2, group_size):
    """
    Function to produce Beaver triple shares.
    """
    U, V = random.randint(3, 6), random.randint(3, 6)
    W = U * V
    return share(U, group_size), share(V, group_size), share(W, group_size), share(beta1, group_size), share(beta2, group_size)

def SPDZ_execute(preparation_data, group_size):
    """
    Function to execute the SPDZ protocol.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        barrier = Barrier(group_size)
        U_shares, V_shares, W_shares, x_shares, y_shares = preparation_data
        queues = [queue.Queue() for _ in range(group_size)]
        futures = [
            executor.submit(
                CP, j, x_shares[j], y_shares[j], U_shares[j], V_shares[j], W_shares[j], 
                queues[j], [queues[k] for k in range(group_size) if k != j], barrier
            ) for j in range(group_size)
        ]

        group_results = [f.result() for f in futures]
        print("Final results share of each party:", group_results)
        sum_total = sum(group_results)
        print("Final Result:", sum_total)

# Main execution
if __name__ == "__main__":
    input1, input2 = 20, 10  # The multiplication input numbers
    client_num = 5  # Number of clients

    preparation_data = SPDZ_prepare(input1, input2, client_num)
    SPDZ_execute(preparation_data, client_num)
